In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random

In [2]:
torch.__version__

'1.1.0'

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [5]:
import sys

In [6]:
sys.version_info

sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)

In [7]:
from mnist import *

In [8]:
# parameters
learning_rate = 0.001
training_epochs = 2
batch_size = 100
drop_prob = 0.3

In [9]:
mymnist = MyMNIST(batch_size)

In [10]:
class DropoutModel(nn.Module):
    def __init__(self, drop_prob):
        super().__init__()
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Linear(784, 512, bias=True),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=drop_prob)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Linear(512, 512, bias=True),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=drop_prob)
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Linear(512, 512, bias=True),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=drop_prob)
        )
        self.layer4 = torch.nn.Sequential(
            torch.nn.Linear(512, 512, bias=True),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=drop_prob)
        )
        self.layer5 = torch.nn.Sequential(
            torch.nn.Linear(512, 10, bias=True),
        )
        
        torch.nn.init.xavier_uniform_(self.layer1[0].weight)
        torch.nn.init.xavier_uniform_(self.layer2[0].weight)
        torch.nn.init.xavier_uniform_(self.layer3[0].weight)
        torch.nn.init.xavier_uniform_(self.layer4[0].weight)
        torch.nn.init.xavier_uniform_(self.layer5[0].weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        return out
        

In [11]:
dropout_model = DropoutModel(drop_prob).to(device)

In [12]:
dropout_model

DropoutModel(
  (layer1): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3)
  )
  (layer2): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3)
  )
  (layer3): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3)
  )
  (layer4): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3)
  )
  (layer5): Sequential(
    (0): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [13]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(dropout_model.parameters(), lr=learning_rate)

In [14]:
total_batch = len(mymnist.train_data_loader)
dropout_model.train()    # set the model to train mode (dropout=True)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in mymnist.train_data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = dropout_model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.311770499
Epoch: 0002 cost = 0.142524689
Learning finished


In [15]:
# Test model and check accuracy
with torch.no_grad():
    dropout_model.eval()    # set the model to evaluation mode (dropout=False)

    # Test the model using test sets
    X_test = mymnist.mnist_test.data.view(-1, 28 * 28).float().to(device)
    Y_test = mymnist.mnist_test.targets.to(device)

    prediction = dropout_model(X_test)
    # print(prediction.shape)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mymnist.mnist_test) - 1)
    X_single_data = mymnist.mnist_test.data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mymnist.mnist_test.targets[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = dropout_model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9722999930381775
Label:  8
Prediction:  3
